In [7]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../')

# import embedding methods
from embedding.bernoulli import Bernoulli
from embedding.KL import KL

# import evaluation methods
from evaluation import evaluate_link_prediction

# import utils
from utils import graph_util
from utils import plot_util
from utils import model_util

# visualization
%matplotlib inline
#sns.set_style('whitegrid')

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Experiment 1

In [10]:
# Experiment 1

exp = {
    "max_epochs": 500,
    "learning_rate": 1e-2, #Adam
    "weight_decay": 1e-7,
    
    "link_prediction": True,
    "link_pred_num_rounds": 3,
    "link_pred_train_ratio": 0.8,
    "link_pred_eval_every_n_steps": 25,
    "link_pred_edge_emb_method": "average",
    
    "node_classification": False,
    "node_class_num_rounds": 3,
    "node_class_train_ratio": 0.8,
    "node_class_eval_every_n_steps": 25,
}

# pick datasets
datasets = ["conra", "hvr"]

model_1 = Bernoulli(embedding_dimension=64, distance_meassure='sigmoid')
model_2 = KL(embedding_dimension=64)

embedding_methods = [model_1, model_2]

# setup folders to store experiment setup summary and results
result_folder = plot_util.setup_folders_and_summary_files(exp, datasets, embedding_methods)
print(f'The results of the current experiment are stored at experiments/{result_folder}')

for dataset in datasets:
    
    # load dataset
    A, y = graph_util.load_dataset(dataset)
    
    for model in embedding_methods:
        
        # do link prediction
        link_prediction_folder = result_folder + "/link_prediction"
        evaluate_link_prediction.expLP(A,dataset,model,exp["link_pred_num_rounds"],
                                       link_prediction_folder, train_ratio=exp["link_pred_train_ratio"], 
                                       edge_emb_method=exp["link_pred_edge_emb_method"],train_epochs=exp["max_epochs"],
                                       eval_epochs=exp["link_pred_eval_every_n_steps"], undirected=True)
        
        # do node classification
#         node_classification_folder = result_folder + "/node_classification"
#         evaluate_node_classification.expNC(A,dataset,embedding_method,exp["number_rounds_node_classification"],node_classification_folder, train_ratio=exp["node_classification_train_ratio"],undirected=True)

The results of the current experiment are stored at experiments/results/2019_12_02_13_42
	Link prediction evaluation has started...
	Link prediction evaluation has started...


AttributeError: 'KL' object has no attribute 'reset_epoch'